## T4 - Scenario plotting

In T4, we will build on our T3 exercises and learn how to plot the intervention scenarios we've built. Note: We intend that these tutorials be done in order, for the greatest benefit. If you haven't yet done T1, T2, and T3, we recommend you start at the beginning.  

First, let's run the scenarios we ended up with in T3. If you're coming here directly from T3, you can skip this step. 

In [ ]:
import fpsim as fp

n_agents   = 10_000
start_year = 1980
repeats    = 3
year       = 2012
youth_ages = ['<18', '18-20']
    
pars = fp.pars(location = 'senegal', n_agents=n_agents, start_year=start_year, end_year=2020) #set pars to include all shared arguments
pars.add_method(name='new injectables', eff=0.983)

method = 'Injectables'
kw = dict(method=method, ages=youth_ages) #set kwargs to take on the method and the age group
d_kw = dict(dest=method, ages = youth_ages)
f1 = fp.make_scen(
        label = '2x uptake',
        year = year,
        probs = [
            dict(init_factor = 2.0, **kw),
                ]
            )
f2 = fp.make_scen(
        label = '5x uptake',
        year = year,
        probs = [
            dict(init_factor = 5.0, **kw),
                ]
            )
    
f3 = fp.make_scen(
        label = '10x uptake',
        year = year,
        probs = [
            dict(init_factor = 10.0, **kw),
                ]
            )
f_switch = fp.make_scen(
        label = '20 percent switching',
        year = year,
        probs = [
            dict(source = 'Injectables', value = 0.20, **d_kw)
                ]
           )

f4 = f2 + f_switch 

method = 'new injectables'
kw = dict(method=method, ages = youth_ages)
f_new = fp.make_scen(
        label = 'introduce new method',
        year = 2015, #replacing the intervention year, giving the new method a bit of a lag
        probs = [
            dict(copy_from='Injectables', **kw),
            dict(init_value=0.05, **kw),
                ]
            )

scens = fp.Scenarios(pars=pars, repeats=repeats)
scens.add_scen(label='Baseline')
scens.add_scen(f1)
scens.add_scen(f2)
scens.add_scen(f3)
scens.add_scen(f4, label = '5x uptake plus switching') #combining f2 and f_switch and re-labeling here
scens.add_scen(f_new)

scens.run()

Now that we've run the scenarios, we have a 'scens' object that we want to examine, and we'll do that first by plotting the results. The default plotting is simply plot(), which will show mCPR, live births, stillbirths, maternal deaths, infant deats, and the infant mortality rate. If you have more than one 'repeat', you'll see each scenario has a line and a confidence band. 

In [ ]:
scens.plot()

Plot() has a couple of additional options beyond the default. 

* <i>'cpr'</i> will plot three different ways to define contraceptive prevalence - mCPR, CPR (includes traditional), and aCPR (includes traditional and restricts denominator to sexually active non-pregnant women) <br>
* <i>'apo'</i> will plot adverse pregnancy outcomes, including abortion and miscarriage <br>
* <i>'mortality'</i> will plot mortality-related outcomes <br>
* <i>'method'</i> plots the method mix, including any new methods you've introduced in the scenarios.  <br>

Let's take a look at the 'method' option. 

In [ ]:
scens.plot('method')

## Customize plotting

Users can customize FPsim plots by contributing new code defining new plot options to our shared codebase. However, if you only want to customize the appearance of existing plots, you can easily add a few lines of code after you've run your scenarios. 

You may want to zoom in on some plots, for instance, to the point of intervention, which you can do in iptyhon's interactive setting. Or you can limit the x-axis when you call the plot. This is particularly helpful if you're calling multiple plots and you want them all to match, without too much fiddling required on your end.

To do this, we simply need to save the figures and write a quick for loop to set the x-axis limits. 

In [ ]:
method_fig = scens.plot('method')
apo_fig = scens.plot('apo')
for fig in [method_fig, apo_fig]:
                for ax in fig.axes:
                    ax.set_xlim(left = 2012, 
                                right = 2020)

You may also want to set custom color schemes for your plots. In Python, you can input HTML color codes into a custom color dictionary: 

In [ ]:
colors = {
    'None'              : [0.0, 0.0, 0.0],
    'Withdrawal'        : [0.3, 0.3, 0.3],
    'Other traditional' : [0.5, 0.5, 0.5],
    'Condoms'           : [0.7, 0.7, 0.7],
    'Pill'              : [0.3, 0.8, 0.9],
    'Injectables'       : [0.6, 0.4, 0.9],
    'Implants'          : [0.4, 0.2, 0.9],
    'IUDs'              : [0.0, 0.0, 0.9],
    'BTL'               : [0.8, 0.0, 0.0],
    'Other modern'      : [0.8, 0.5, 0.5],
    'new condoms'       : [0.7, 0.8, 0.7],
    'new implants'      : [0.5, 0.8, 0.5],
    'new injectables'   : [0.2, 0.8, 0.2],
}

Then we can apply the color dictionary to our method mix plot: 

In [ ]:
scens.plot('method', colors = colors)

This concludes the FPsim tutorials. We look forward to seeing what you create using this tool!